### 2023/02/10
### すべての年度の学習者のデータを取得する

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import bs4
import re
import glob
import os
from func_graph import *
from func_process import *
URL = 'http://skill.shinshu-u.ac.jp/2015'
# すべての年度と月と日付
ymd = pd.read_excel('../temp/date_all_member.xlsx')

#### ▶すべてのURLとモチーフ，名前，回数の情報を取得
##### ▶url_ymdmtnの構成↓
|  ドローイングクラス実施順  |  その日のURLのリスト  |  その日のドローイング情報のリスト  |
| ---- | ---- | ---- |
|  1  |  [ 井原さん紙箱1回目のURL,  高田さん紙箱1回目のURL, ... ]  |  [ (2013，4，18，紙箱，1，井原さん), (2013，4，18，紙箱，1，高田さん),... ]  |
|  2  |  [ 井原さん紙箱2回目のURL,  高田さん紙箱2回目のURL, ... ]  |  [ (2013，5，13，紙箱，2，井原さん), (2013，5，13，紙箱，2，高田さん),... ]  |
|  :  | : | : |
| 58  |  [ 井原さん紙箱1回目のURL,  高田さん紙箱1回目のURL, ... ]  |  [ (2021，7，19，紙袋，3，松尾さん), (2021，7，19，紙袋，3，村岡さん),... ]  |

In [2]:
def get_all_url_date(year, month, day):
    url = f'http://skill.shinshu-u.ac.jp/{year}/{month}-{day}'
    res = requests.get(url, auth=('19t2003a', 'ireneRED77'))
    soup = bs4.BeautifulSoup(res.text, "html.parser")
    portfolio_area = soup.find('div', id='portfolioArea')
    
    # URLの取得
    list_a = portfolio_area.select('a')
    list_url = [ a.get('href') + '/strokes/pointlists' for a in list_a ]
   
    # モチーフ名・回数・名前を取得
    thumbnail = portfolio_area.find_all('div', class_='thumbnail')
    year_month_day_motif_times_name = []
    for item in thumbnail:
        motif_times = item.find('p').find_all('span')[0].string
        motif = re.sub(r'[0-9]+', '', motif_times)
        times = re.sub(r'\D', '', motif_times)
        name = item.find('p').find_all('span')[1].string.replace('　', '')
        year_month_day_motif_times_name.append((year, month, day, motif, times, name))
        
    return list_url, year_month_day_motif_times_name

url_ymdmtn = []

for y, m, d in zip(ymd['year'], ymd['month'], ymd['day']):
    list_url, year_month_day_motif_times_name = get_all_url_date(y, m, d)
    url_ymdmtn.append([list_url, year_month_day_motif_times_name])

url_ymdmtn

[[['http://skill.shinshu-u.ac.jp/2013/4-18/20130417141007_18-4-0-0-0_strokes.txt/strokes/pointlists',
   'http://skill.shinshu-u.ac.jp/2013/4-18/20130417140919_18-4-0-0-0_strokes.txt/strokes/pointlists',
   'http://skill.shinshu-u.ac.jp/2013/4-18/20130417140816_18-4-0-0-0_strokes.txt/strokes/pointlists',
   'http://skill.shinshu-u.ac.jp/2013/4-18/20130417140951_18-4-0-0-0_strokes.txt/strokes/pointlists',
   'http://skill.shinshu-u.ac.jp/2013/4-18/20130417140933_18-4-0-0-0_strokes.txt/strokes/pointlists',
   'http://skill.shinshu-u.ac.jp/2013/4-18/20130417140717_18-4-0-0-0_strokes.txt/strokes/pointlists',
   'http://skill.shinshu-u.ac.jp/2013/4-18/20130417140800_18-4-0-0-0_strokes.txt/strokes/pointlists',
   'http://skill.shinshu-u.ac.jp/2013/4-18/20130417140733_18-4-0-0-0_strokes.txt/strokes/pointlists',
   'http://skill.shinshu-u.ac.jp/2013/4-18/20130417121237_18-4-0-0-0_strokes.txt/strokes/pointlists',
   'http://skill.shinshu-u.ac.jp/2013/4-18/20130417140640_18-4-0-0-0_strokes.txt/s

#### ▶年ごとのフォルダの作成

In [20]:
# # フォルダを作成したい場所
# output = '../temp/data_all_year'
# for y, m, d in zip(ymd['year'], ymd['month'], ymd['day']):
#     os.makedirs(f'{output}/{y}', exist_ok=True)

#### ▶URLと日付，モチーフ，名前，回数の情報をpickleファイルとして保存

In [ ]:
url_all = []
ymdmtn_all = []
for row in url_ymdmtn:
    url_all += row[0]
    ymdmtn_all += row[1]
df_url_ymdmtn = pd.DataFrame({'url': url_all, 'ymdmtn': ymdmtn_all})
df_url_ymdmtn.to_excel('../temp/all_url_ymdmtn.xlsx')
df_url_ymdmtn.to_pickle('../temp/all_url_ymdmtn.pkl')

#### ▶日付や回数などの表記方法の確認

In [45]:
df_url_ymdmtn = pd.read_pickle('../temp/all_url_ymdmtn.pkl')
create_html_from_df(df_url_ymdmtn)

url = [ url for url in df_url_ymdmtn['url'] ]
year = [ row[0] for row in df_url_ymdmtn['ymdmtn'] ]
month = [ row[1] for row in df_url_ymdmtn['ymdmtn'] ]
day = [ row[2] for row in df_url_ymdmtn['ymdmtn'] ]
motif = [ row[3] for row in df_url_ymdmtn['ymdmtn'] ]
times = [ row[4] for row in df_url_ymdmtn['ymdmtn'] ]
name = [ row[5] for row in df_url_ymdmtn['ymdmtn'] ]

df_url_ymdmtn = pd.DataFrame({'url': url, 'year': year, 'month': month, 'day': day, 'motif': motif, 'times': times, 'name': name})
df_url_ymdmtn.nunique()

print('motif : ', df_url_ymdmtn['motif'].unique())
print('times : ', df_url_ymdmtn['times'].unique())
df_url_ymdmtn.to_excel('../temp/all_url_ymdmtn.xlsx')

motif :  ['箱' 'スタバの紙袋' '紙袋' '鉛筆削り' '紙箱' '紙箱-' '紙箱と紙袋' '箱-']
times :  ['' '1' '2' '3' '22' '0']


In [53]:
def get_unique_df(df, column, element):
    df_copy = df.copy()
    df_unique = df_copy[ df_copy[column] == element ]
    return df_unique

df_times_null = get_unique_df(df_url_ymdmtn, 'times', '')
df_times_22 = get_unique_df(df_url_ymdmtn, 'times', '22')
df_times_0 = get_unique_df(df_url_ymdmtn, 'times', '0')

df_motif_box = get_unique_df(df_url_ymdmtn, 'motif', '箱')
df_motif_paperbox = get_unique_df(df_url_ymdmtn, 'motif', '紙箱')
df_motif_paperbox = get_unique_df(df_url_ymdmtn, 'motif', '紙箱-')


df_year_2016 = get_unique_df(df_url_ymdmtn, 'year', 2016)

df_view = get_unique_df(df_url_ymdmtn, 'motif', '紙袋')
create_html_from_df(df_view)

#### ▶自力で「motif」列を修正した後のexcelファイルを読み込む
#### ▶&「times」列の型を修正

In [78]:
df_modify = pd.read_excel('../temp/all_url_ymdmtn_modify.xlsx')
del df_modify['Unnamed: 0']
# 欠損値の確認
# df_modify.isna().any()
df_times_fillna = df_modify.fillna(9)
df_times_fillna['times'].unique()
df_times_fillna.to_excel('../temp/all_url_ymdmtn_times_modify.xlsx')

#### ▶「times」を自力で修正した後のexcelファイルを読み込む

In [89]:
df_url_ymdmtn = pd.read_excel('../data/all_url_ymdmtn_modify_complete.xlsx', index_col=0)
df_url_ymdmtn

,url,year,month,day,motif,times,name
0,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,2013,4,18,紙箱,1,和井田瞳
1,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,2013,4,18,紙箱,1,長田美穂子
2,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,2013,4,18,紙箱,1,佐藤宏哉
3,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,2013,4,18,紙箱,1,井原亜利紗
4,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,2013,4,18,紙箱,1,山田美沙紀
...,...,...,...,...,...,...,...
950,http://skill.shinshu-u.ac.jp/2021/7-19/2021071...,2021,7,19,紙袋,3,川﨑朱音
951,http://skill.shinshu-u.ac.jp/2021/7-19/2021071...,2021,7,19,紙袋,3,望月創太
952,http://skill.shinshu-u.ac.jp/2021/7-19/2021071...,2021,7,19,紙袋,3,松尾俊佑
953,http://skill.shinshu-u.ac.jp/2021/7-19/2021071...,2021,7,19,紙袋,3,村岡杏菜


#### ▶URLからTSVファイルを作成

In [2]:
# urlからtsvファイルの作成
data = pd.read_excel('../data/all_url_ymdmtn_modify_complete.xlsx', index_col=0)[600:]
output_folder = '../temp/data_all_year'
count = 0
all_count = len(data)

for url, year, month, day, motif, times, name in zip(data['url'], data['year'], data['month'], data['day'], data['motif'], data['times'], data['name']):
    if motif == '紙箱':
        motif_eng = 'box'
    elif motif == '紙袋':
        motif_eng = 'bag'
    else:
        motif_eng = 'exc'
    
    file_name = f'{output_folder}/{year}/{year}_{month}_{day}_{motif_eng}_{times}_{name}_org.tsv'
    print(file_name)
    df = get_df_from_url(url)
    df.to_csv(file_name, sep='\t')
    
    # count += 1
    # print(count, '/', all_count)
    
    
print('完了!')

../temp/data_all_year/2017/2017_6_20_bag_2_佐藤文香_org.tsv
ok
../temp/data_all_year/2017/2017_6_20_bag_2_志村連_org.tsv
ok
../temp/data_all_year/2017/2017_6_20_bag_2_相田詩苑_org.tsv
ok
../temp/data_all_year/2017/2017_6_20_bag_2_小澤祐希奈_org.tsv
ok
../temp/data_all_year/2017/2017_6_20_bag_2_佐藤志保_org.tsv
ok
../temp/data_all_year/2017/2017_6_20_bag_2_角田晴彦_org.tsv
ok
../temp/data_all_year/2017/2017_6_20_bag_2_白玉海史_org.tsv
ok
../temp/data_all_year/2017/2017_6_20_bag_2_安川友貴_org.tsv
ok
../temp/data_all_year/2017/2017_6_20_bag_2_小田綾乃_org.tsv
ok
../temp/data_all_year/2017/2017_6_20_bag_2_上床桜子_org.tsv
ok
../temp/data_all_year/2017/2017_6_20_bag_2_遠藤駿仁_org.tsv
ok
../temp/data_all_year/2017/2017_7_11_bag_3_安川友貴_org.tsv
ok
../temp/data_all_year/2017/2017_7_11_bag_3_山中将靖_org.tsv
ok
../temp/data_all_year/2017/2017_7_11_bag_3_相田詩苑_org.tsv
ok
../temp/data_all_year/2017/2017_7_11_bag_3_小澤祐希奈_org.tsv
ok
../temp/data_all_year/2017/2017_7_11_bag_3_小田綾乃_org.tsv
ok
../temp/data_all_year/2017/2017_7_11_bag_3_佐藤文香_org.tsv

#### ▶TSVファイルからPickleファイルの生成

In [57]:
year_folder = 2021
tsv_folder = f'../data/data_all_year_tsv/{year_folder}'
output_folder = f'../data/data_all_year_pkl/{year_folder}'


tsv_files = glob.glob(f'{tsv_folder}/*')
all_count = len(tsv_files)
count = 0

for file in tsv_files:
    df = pd.read_csv(file, delimiter='\t')
    file_name = file.replace('.tsv', '')
    file_name = file_name.replace(f'{tsv_folder}', '')
    df_pkl = create_pkl_from_df(df)
    df_pkl.to_pickle(f'{output_folder}/{file_name}.pkl')
    
    count += 1
    print(count, '/', all_count, ' | ', file_name)



1 / 110  |  \2021_4_19_box_0_中西鈴_org
2 / 110  |  \2021_4_19_box_0_伊藤紫歩_org
3 / 110  |  \2021_4_19_box_0_加藤千野_org
4 / 110  |  \2021_4_19_box_0_安部琴海_org
5 / 110  |  \2021_4_19_box_0_小川日菜子_org
6 / 110  |  \2021_4_19_box_0_川﨑朱音_org
7 / 110  |  \2021_4_19_box_0_村岡杏菜_org
8 / 110  |  \2021_4_19_box_0_松尾俊佑_org
9 / 110  |  \2021_4_19_box_0_瀧澤一平_org
10 / 110  |  \2021_4_19_box_0_荒金美帆_org
11 / 110  |  \2021_4_19_box_0_菅井雄梧_org
12 / 110  |  \2021_4_19_box_0_野原詩音_org
13 / 110  |  \2021_4_19_box_0_門井李々花_org
14 / 110  |  \2021_4_19_box_0_首藤桜音_org
15 / 110  |  \2021_4_19_box_0_黒部咲_org
16 / 110  |  \2021_5_10_box_1_伊藤紫歩_org
17 / 110  |  \2021_5_10_box_1_加藤千野_org
18 / 110  |  \2021_5_10_box_1_安部琴海_org
19 / 110  |  \2021_5_10_box_1_小川日菜子_org
20 / 110  |  \2021_5_10_box_1_川﨑朱音_org
21 / 110  |  \2021_5_10_box_1_村岡杏菜_org
22 / 110  |  \2021_5_10_box_1_松尾俊佑_org
23 / 110  |  \2021_5_10_box_1_瀧澤一平_org
24 / 110  |  \2021_5_10_box_1_皆川柚羽_org
25 / 110  |  \2021_5_10_box_1_荒金美帆_org
26 / 110  |  \2021_5_10_box_1_菅井雄

### ▶回転の方向を1に統一してあるので、目視で回転の方向を調べる!

In [67]:
year_pkl = 2021
pkl_files = glob.glob(f'../data/data_all_year_pkl/{year_pkl}/*')
output_folder = f'../data/data_all_year_pkl_addparams/{year_pkl}'
# 回転の方向の種類を1に統一
direct_type = [ 1 for i in range(len(pkl_files)) ]

for file, dir in zip(pkl_files, direct_type):
    df = pd.read_pickle(file)
    df_add_params = add_params_to_original(df, dir)
    file_name = file.replace('_org.pkl', '')
    file_name = file_name.replace(f'../data/data_all_year_pkl/{year_pkl}', '')
    df_add_params.to_pickle(f'{output_folder}/{file_name}_addparams.pkl')
    print(file_name)
    

\2021_4_19_box_0_中西鈴
\2021_4_19_box_0_伊藤紫歩
\2021_4_19_box_0_加藤千野
\2021_4_19_box_0_安部琴海
\2021_4_19_box_0_小川日菜子
\2021_4_19_box_0_川﨑朱音
\2021_4_19_box_0_村岡杏菜
\2021_4_19_box_0_松尾俊佑
\2021_4_19_box_0_瀧澤一平
\2021_4_19_box_0_荒金美帆
\2021_4_19_box_0_菅井雄梧
\2021_4_19_box_0_野原詩音
\2021_4_19_box_0_門井李々花
\2021_4_19_box_0_首藤桜音
\2021_4_19_box_0_黒部咲
\2021_5_10_box_1_伊藤紫歩
\2021_5_10_box_1_加藤千野
\2021_5_10_box_1_安部琴海
\2021_5_10_box_1_小川日菜子
\2021_5_10_box_1_川﨑朱音
\2021_5_10_box_1_村岡杏菜
\2021_5_10_box_1_松尾俊佑
\2021_5_10_box_1_瀧澤一平
\2021_5_10_box_1_皆川柚羽
\2021_5_10_box_1_荒金美帆
\2021_5_10_box_1_菅井雄梧
\2021_5_10_box_1_野原詩音
\2021_5_10_box_1_門井李々花
\2021_5_10_box_1_首藤桜音
\2021_5_10_box_1_黒部咲
\2021_5_24_box_2_中西鈴
\2021_5_24_box_2_伊藤紫歩
\2021_5_24_box_2_加藤千野
\2021_5_24_box_2_安部琴海
\2021_5_24_box_2_小川日菜子
\2021_5_24_box_2_川﨑朱音
\2021_5_24_box_2_村岡杏菜
\2021_5_24_box_2_松尾俊佑
\2021_5_24_box_2_瀧澤一平
\2021_5_24_box_2_皆川柚羽
\2021_5_24_box_2_荒金美帆
\2021_5_24_box_2_菅井雄梧
\2021_5_24_box_2_野原詩音
\2021_5_24_box_2_門井李々花
\2021_5_24_box_2_首藤桜音
\2021_5_

In [68]:
df_test = pd.read_pickle('../data/data_all_year_pkl_addparams/2013/2013_4_18_box_1_井原亜利紗_addparams.pkl')
df_test

,Unnamed: 0,time,alpha,max_points,ptx,pty,type_str,pressure_style,length_mm_acc,length_anoto,pressure_avg,type_int,init_ptx,init_pty,polar_radius,polar_degree,pre_init_vector,pre_init_dist,pre_init_unit_vector
0,0,0,233.696742,19,"[-106.83749999999998, -105.67500000000001, -10...","[-45.900000000000006, -46.2, -46.6125, -47.100...",Straight,64.959836,80.054154,266.847181,115.473684,1,-106.8375,-45.9000,116.280099,-156.750458,"[0, 0]",0.000000,"[0, 0]"
1,1,2,216.172840,27,"[-40.912499999999994, -41.17499999999998, -41....","[-89.4, -89.73750000000001, -89.92500000000001...",Straight,96.504073,317.145273,790.303730,106.814815,1,-40.9125,-89.4000,98.316797,-114.590454,"[65.92499999999998, -43.5]",78.983262,"[0.8346705188034002, -0.5507496028509354]"
2,2,9,221.190476,17,"[-40.79999999999998, -40.79999999999998, -40.8...","[-88.0875, -88.1625, -90.0, -91.4625, -92.625,...",Simple curve,91.360330,358.455194,137.699737,109.294118,2,-40.8000,-88.0875,97.077534,-114.852431,"[0.11250000000001137, 1.3125]",1.317313,"[0.08540114134644437, 0.9963466490417503]"
3,3,11,239.177489,22,"[-106.57499999999999, -106.46249999999998, -10...","[-45.825, -47.775000000000006, -48.97500000000...",Simple curve,54.695161,396.745467,127.634241,118.181818,2,-106.5750,-45.8250,116.009315,-156.733316,"[-65.775, 42.2625]",78.182284,"[-0.8413031269476842, 0.5405636397206613]"
4,4,14,193.948413,12,"[-107.58749999999998, -107.58749999999998, -10...","[-83.3625, -83.4, -83.7, -86.0625, -86.7375000...",Simple curve,91.025615,402.613360,19.559644,95.833333,2,-107.5875,-83.3625,136.104286,-142.230243,"[-1.0124999999999886, -37.537499999999994]",37.551153,"[-0.02696322028934839, -0.9996364262828902]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038,1038,1199,247.469546,43,"[-96.11250000000001, -90.75, -89.0999999999999...","[-90.825, -95.1375, -96.67500000000001, -98.4,...",Complex,56.887463,107349.937110,375.301293,122.279070,3,-96.1125,-90.8250,132.237639,-136.620171,"[-5.0625, -22.799999999999997]",23.355276,"[-0.21676044633569222, -0.9762248249785249]"
1039,1039,1200,235.384615,26,"[-93.52499999999998, -94.3125, -94.61250000000...","[-68.175, -70.425, -73.0125, -82.3500000000000...",Complex,49.920633,107457.105874,357.229213,116.307692,3,-93.5250,-68.1750,115.735717,-143.909841,"[2.587500000000034, 22.650000000000006]",22.797317,"[0.11350019896225796, 0.9935379735246801]"
1040,1040,1204,186.190476,1,[-89.09999999999997],[-74.7375],Point,0.000000,107457.105874,0.000000,92.000000,0,-89.1000,-74.7375,116.294901,-140.009917,"[4.425000000000011, -6.5625]",7.914988,"[0.5590659308461626, -0.8291232025260865]"
1041,1041,1204,246.259129,163,"[-88.80000000000001, -88.83749999999998, -89.0...","[-72.675, -71.925, -71.66250000000001, -71.887...",Complex,83.875145,108007.509653,1834.679263,121.680982,3,-88.8000,-72.6750,114.747966,-140.702688,"[0.2999999999999545, 2.0625]",2.084204,"[0.1439398457140894, 0.9895864392845146]"


In [33]:
year_view = 2013
files = glob.glob('../data/data_all_year_pkl_addparams/{year_view}')
for file in files:
    df = pd.read_pickle(file)
    print('=====', file, '==================================')
    draw_stroke(df)

In [ ]:
file_box = glob.glob('../data/2015/pickle/box_addparams/*')
df_box = [ pd.read_pickle(file) for file in file_box]

file_bag = glob.glob('../data/2015/pickle/bag_addparams/*')
df_bag = [ pd.read_pickle(file) for file in file_bag]